In [24]:
import pandas as pd
import numpy as np

In [25]:
LOW = 0.08
HIGH = 0.85

In [26]:
#load prediction file for both detection and classification
det = pd.read_csv('../checkpoint/3003/tester.csv')
cls = pd.read_csv('../checkpoint/1703/classifier/default.csv')

In [27]:
print(det.shape)
det.head()

(3029, 2)


,image_id,PredictionString
0,83caa8a85e03606cf57e49147d7ac569,3 0.86 809 1131 1809 1647 0 0.22 1065 741 1325...
1,7550347fa2bb96c2354a3716dfa3a69c,0 0.64 1239 737 1528 1055 3 0.42 791 1622 1898...
2,74b23792db329cff5843e36efb8aa65a,10 0.31 160 2508 262 2644 11 0.28 160 2507 261...
3,94568a546be103177cb582d3e91cd2d8,3 0.83 635 1628 1512 1936 0 0.37 914 1016 1216...
4,6da36354fc904b63bc03eb3884e0c35c,0 0.51 1082 543 1300 766 13 0.36 1447 720 1692...


In [28]:
print(cls.shape)
cls.head()

(3000, 2)


,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0.939700
3,009bc039326338823ca3aa84381f17f1,0.123799
4,00a2145de1886cb9eb88869c85d74080,0.654006


In [29]:
final_pred = pd.merge(det, cls, on = 'image_id', how = 'left')
print(final_pred.shape)
final_pred.head()

(3029, 3)


,image_id,PredictionString,target
0,83caa8a85e03606cf57e49147d7ac569,3 0.86 809 1131 1809 1647 0 0.22 1065 741 1325...,0.970583
1,7550347fa2bb96c2354a3716dfa3a69c,0 0.64 1239 737 1528 1055 3 0.42 791 1622 1898...,0.039873
2,74b23792db329cff5843e36efb8aa65a,10 0.31 160 2508 262 2644 11 0.28 160 2507 261...,0.010240
3,94568a546be103177cb582d3e91cd2d8,3 0.83 635 1628 1512 1936 0 0.37 914 1016 1216...,0.065679
4,6da36354fc904b63bc03eb3884e0c35c,0 0.51 1082 543 1300 766 13 0.36 1447 720 1692...,0.838772


In [30]:
#Number of 'No Findings' before filtering
final_pred['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    30
Name: PredictionString, dtype: int64

In [31]:
def filter_class(row, low_thr=LOW, high_thr=HIGH):
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif low_thr<=prob<high_thr:
        ## More chance of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [32]:
submission = final_pred.apply(filter_class, axis=1)
submission.head()

,image_id,PredictionString,target
0,83caa8a85e03606cf57e49147d7ac569,3 0.86 809 1131 1809 1647 0 0.22 1065 741 1325...,0.970583
1,7550347fa2bb96c2354a3716dfa3a69c,14 1 0 0 1 1,0.039873
2,74b23792db329cff5843e36efb8aa65a,14 1 0 0 1 1,0.010240
3,94568a546be103177cb582d3e91cd2d8,14 1 0 0 1 1,0.065679
4,6da36354fc904b63bc03eb3884e0c35c,0 0.51 1082 543 1300 766 13 0.36 1447 720 1692...,0.838772


In [33]:
#Number of 'No Findings' after filtering
submission['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    1941
Name: PredictionString, dtype: int64

In [34]:
submission[['image_id', 'PredictionString']].to_csv('../checkpoint/submission_ensemble.csv',index = False)